In [ ]:
import numpy as np
import os
import seaborn as sns
import pandas as pd
from math import e,sqrt
from pyFTS.fcm import Activations
from pyFTS.models import hofts
from numpy.linalg import svd
import matplotlib.pyplot as plt
from pyFTS.fcm import fts as fcm_fts
from pyFTS.partitioners import Grid,CMeans
from pyFTS.common import Util
from pyFTS.common import Membership as mf
from scipy.optimize import least_squares
from scipy.optimize import leastsq
from pyFTS.benchmarks import Measures
from pyFTS.common import Util
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import StandardScaler, KernelCenterer, MinMaxScaler
from sklearn.utils import extmath
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics.pairwise import euclidean_distances
import datetime


In [ ]:
import numpy as np
from math import e,sqrt

def step(x, deriv=False):
    if deriv:
        return np.abs(1 * (x == 0))
    else:
        return np.abs(1 * (x > 0))


def sigmoid(x, deriv=False):
    if deriv:
        #return sigmoid(x)*(1 - sigmoid(x))
        return x * (1 - x)
    else:
        return 1 / (1 + np.exp(-x))


def softmax(x, deriv=False):
    if deriv:
        pass
    else:
        mvs = sum([np.exp(k) for k in x.flatten()])
        return np.array([np.exp(k)/mvs for k in x.flatten()])


def tanh(x, deriv=False):
    if deriv:
        return 1 - np.tanh(x)**2
    else:
        return np.tanh(x)


def relu(x, deriv=False):
    if deriv:
        return np.abs(1. * (x > 0))
    else:
        return np.abs(x * (x > 0))


# SoftPlus
def softplus(x,deriv=False):
  if deriv:
    return 1/(1+((e)**-x))
  else:
    return np.log(1+((e)**x))

# Bent identity:
def bentid(x,deriv=False):
  if deriv:
    return (x/(2*(sqrt((x**2)+1))))+1
  else:
    return (sqrt(((x**2)+1)-1)/2)+x

# SoftExponential:
def softex(x,a=0.01,deriv=False):
  if deriv:
    return 1/(1-a*(a+x)) if a < 0 else (e)**(a*x)
  else:
    if a < 0:
      return -((np.log(1-a*(x+a)))/a)
    elif a == 0:
      return x
    elif a > 0:
      return (((e)**(a*x))/a)+a


# Gaussian:
def gaussian(x,deriv=False):
  if deriv:
    return -2*x*(e)**((-x)**2)
  else:
    return (e)**((-x)**2)




In [ ]:
#Embedding PCA and KPCA
class EmbeddingPCA():
    def __init__(self, no_of_components,gamma):
        self.no_of_components = no_of_components
        self.eigen_values = None
        self.eigen_vectors = None

        # debug attributes
        self.name = 'Principle Component Analysis'
        self.shortname = 'PCA'

        self.std = StandardScaler()
        self.pca_sk = PCA(n_components=self.no_of_components)
        self.gamma = gamma
        self.kpca_sk = KernelPCA(n_components=self.no_of_components, kernel='rbf', gamma=gamma, fit_inverse_transform = True)

    def __str__(self):
        return self.name

    def transform(self, x):
        return np.dot(x - self.mean, self.projection_matrix.T)

    def inverse_transform(self, x):
        return np.dot(x, self.projection_matrix) + self.mean

    def fit(self, x):
        self.no_of_components = x.shape[1] if self.no_of_components is None else self.no_of_components
        self.mean = np.mean(x, axis=0)

        cov_matrix = np.cov(x - self.mean, rowvar=False)

        self.eigen_values, self.eigen_vectors = np.linalg.eig(cov_matrix)
        self.eigen_vectors = self.eigen_vectors.T

        self.sorted_components = np.argsort(self.eigen_values)[::-1]

        self.projection_matrix = self.eigen_vectors[self.sorted_components[:self.no_of_components]]
        self.explained_variance = self.eigen_values[self.sorted_components];
        self.explained_variance_ratio = self.explained_variance / self.eigen_values.sum()

    def pca_sklearn(self,x):
        return self.pca_sk.fit_transform(x);

    def pca_sklearn_inverse(self, x):
        return self.pca_sk.inverse_transform(x);

    def standardization(self, x):
        return self.std.fit_transform(x)

    def standardization_inverse(self,x):
        return self.std.inverse_transform(x)

    def kernel_pca(self,x, gamma):
        # Calculate euclidean distances of each pair of points in the data set
        dist = euclidean_distances(x, x, squared=True)

        # Calculate Kernel matrix
        k = np.exp(-gamma * dist)
        kc = KernelCenterer().fit_transform(k)

        # Get eigenvalues and eigenvectors of the kernel matrix
        eig_vals, eig_vecs = np.linalg.eigh(kc)

        # flip eigenvectors' sign to enforce deterministic output
        eig_vecs, _ = extmath.svd_flip(eig_vecs, np.empty_like(eig_vecs).T)

        # Concatenate the eigenvectors corresponding to the highest n_components eigenvalues
        xkpca = np.column_stack([eig_vecs[:, -i] for i in range(1, self.no_of_components + 1)])

        return xkpca

    def kernel_pca_sklearn(self, x, gamma):
        #self.kpca_sk = KernelPCA(n_components=self.no_of_components, kernel='rbf', gamma=gamma)
        return self.kpca_sk.fit_transform(x)

    def kernel_pca_sklearn_inverse(self, x, gamma):
        return self.kpca_sk.inverse_transform(x)

In [ ]:

class FuzzyCognitiveMap(object):
    def __init__(self, **kwargs):
        super(FuzzyCognitiveMap, self).__init__()
        self.order = kwargs.get('order',1)
        self.concepts = kwargs.get('partitioner',None)
        self.weights = []
        self.bias=[]
        self.activation_function = kwargs.get('activation_function', Activations.sigmoid)
        #self.activation_function=softplus

    def activate(self, memberships):
        dot_products = np.zeros(len(self.concepts))
        for k in np.arange(0, self.order):
            dot_products += np.dot(np.array(memberships[k]).T, self.weights[k])+(self.bias[k])
        return self.activation_function( dot_products )


In [ ]:
def func(values, *arg):
    #print(values.shape)
    if len(values.shape) == 1:
      n_coefs = values.shape[0] + 1
      #return arg[0] + np.sum([values[i-1] * arg[i] for i in np.arange(1,n_coefs)])
      ret = 0
    else:
      n_coefs = values.shape[0] + 1
      n_inst = values.shape[1]
      ret = np.zeros(n_inst)
    for k in range(1, n_coefs):
      ret = ret + arg[k] * values[k-1]

    ret = ret + arg[0]

    return ret



In [ ]:
from pyFTS.models import hofts
from numpy.linalg import svd

def func2(x, a, b, c):
    z1,z2 = x
    return a + (b * z1) + (c * z2)

def func3(x, a, b, c, d):
    z1,z2,z3 = x
    return a + (b * z1) + (c * z2) + (d * z3)

def func4(x, a, b, c, d, e):
    z1,z2,z3,z4 = x
    return a + (b * z1) + (c * z2) + (d * z3) + (e * z4)


class FCM_FTS(hofts.HighOrderFTS):

    def __init__(self, **kwargs):
        super(FCM_FTS, self).__init__(**kwargs)

        num_concepts = self.partitioner.partitions

        self.num_fcms = kwargs.get('num_fcms', 2)

        self.fcm = []

        self.loss_function = kwargs.get('loss', func)

        for k in range(self.num_fcms):
          fcm_tmp = FuzzyCognitiveMap(**kwargs)
          weights = np.random.uniform(-1, 1, size=(self.order,num_concepts, num_concepts))
          specturalradius1=np.max(np.abs(np.linalg.eigvals(weights)))
          fcm_tmp.weights = weights*0.5/specturalradius1
          bias = np.random.uniform(-1, 1, size=(self.order,num_concepts))
          U,S,VT=svd(bias)
          specturalradius2=np.max(S)
          fcm_tmp.bias=bias*0.5/specturalradius2
          self.fcm.append(fcm_tmp)


        # Coefficients

        #self.theta = np.zeros(self.num_fcms + 1)
        self.theta = np.random.rand(self.num_fcms + 1)

    def forecast(self, data, **kwargs):
        y1 = []

        midpoints = np.array([fset.centroid for fset in self.partitioner])

        for t in np.arange(self.order, len(data)+1):

            sample = data[t - self.order : t]

            fuzzyfied = self.partitioner.fuzzyfy(sample, mode='vector')

            # Evaluate FCMs

            forecasts = []

            for fcm in self.fcm:
                activation=fcm.activate(fuzzyfied)
                forecasts.append(np.dot(midpoints, activation)/np.nanmax([1, np.sum(activation)]))

            # Combine the results

            #print(forecasts)

            result = self.loss_function(np.array(forecasts), *self.theta)

            if str(result) == 'nan' or result == np.nan or result == np.Inf:
               print('error')

            y1.append(result)

        return y1

    def run_fcm(self, fcm, data):
        ret = []
        midpoints = np.array([fset.centroid for fset in self.partitioner])
        for t in np.arange(self.order, len(data)+1):
            sample = data[t - self.order : t]
            fuzzyfied = self.partitioner.fuzzyfy(sample, mode='vector')
            activation = fcm.activate(fuzzyfied)
            final = np.dot(midpoints, activation)/np.nanmax([1, np.sum(activation)])
            ret.append(final)
        return ret

    def train(self, data, **kwargs):
        from scipy.optimize import curve_fit, least_squares, minimize, leastsq

        outputs = []

        for model in self.fcm:
          outputs.append(self.run_fcm(model, data)[:-1])

        f = lambda coef, y, x: self.loss_function(x, *coef) - y

        #xdata = tuple(outputs)

        #self.theta, cov = curve_fit(self.loss_function, xdata=outputs, ydata=data[self.order:], p0=self.theta )
                                       #n=self.num_fcms+1)#,
        self.theta, flag = leastsq(f, x0 = self.theta, args=(data[self.order:], np.array(outputs)))
#         print(self.theta)


In [ ]:
import numpy as np
import pandas as pd

from pyFTS.models.nonstationary import partitioners as nspart
from pyFTS.models.nonstationary import nsfts
from pyFTS.fcm import Activations
class GRHFCM_PCA():
    def __init__(self, num_components_pca, gamma):
        # debug attributes
        self.name = 'Grouped Random High Order FCM-PCA '
        self.shortname = 'GRHFCM_PCA'

        ## Parameters of the model
        self.num_components_pca = num_components_pca
        #self.fts_model = fts_model
        #self.order_fts_model = order_fts_model
        #self.model = self.fts_model
        #self.memory_window_error = memory_window_error
        self.gamma = gamma


        # PCA
        self.pca = EmbeddingPCA(no_of_components=self.num_components_pca, gamma = self.gamma)

    def run_test(self, models, data,transformation):
        if transformation == 'PCA':
            pca_test = self.create_pca_components(data)
        else:
            pca_test = self.create_kernel_pca_components(data)

        components = pca_test.columns
        forecast_model = []

        for i in range(0, self.num_components_pca):
            mf = models[i].predict(np.array(pca_test.loc[:,components[i]]))
            if (i == 0):
                forecast_model = mf
            else:
                forecast_model = np.column_stack((forecast_model,mf))

        if transformation == 'PCA':
            forecast = self.inverse_transformation(forecast_model)
        else:
            forecast = self.inverse_kpca_transformation(forecast_model)

        return forecast, data

    def run_train(self, data,transformation):
        if transformation == 'PCA':
            pca_train = self.create_pca_components(data)
        else:
            pca_train = self.create_kernel_pca_components(data)
        models = self.create_and_fit_mimo_models(pca_train)
        return models,pca_train

    def create_and_fit_mimo_models(self, data):
        components = data.columns
        models = []

        for i in range(0, self.num_components_pca):
            md = self.create_GRHFCM_FTS(data.loc[:,components[i]])
            md.fit(np.array(data.loc[:,components[i]]))
            models.append(md)

        return models

    def create_GRHFCM_FTS(self,data):
        partitioner = Grid.GridPartitioner(data=data, npart=3, mf=mf.trimf)

        model = FCM_FTS(partitioner=partitioner, order=2, num_fcms=10,
                  activation_function=relu,
                  loss=func)


        return model

    def create_pca_components(self,data):
        transformed = self.pca.standardization(data)
        x_std = self.pca.pca_sklearn(transformed)

        components = []
        for i in range(0, self.num_components_pca):
            components.append('C' + str(i))

        df_pca = pd.DataFrame(x_std, columns=list(components))
        return df_pca

    def create_names_compoments(self):
        components = None
        for i in range (0,self.num_components_pca):
            components.append()

    def create_kernel_pca_components(self,data):
        transformed = self.pca.standardization(data)
        x_std = self.pca.kernel_pca_sklearn(transformed,self.gamma)

        components = []
        for i in range(0, self.num_components_pca):
            components.append('C' + str(i))

        df_kpca = pd.DataFrame(x_std, columns=list(components))
        return df_kpca

    def inverse_transformation(self, data):
        inverse_pca = self.pca.pca_sklearn_inverse(data)
        final_result = self.pca.standardization_inverse(inverse_pca)
        return final_result

    def inverse_kpca_transformation(self, data):
        inverse_kpca = self.pca.kernel_pca_sklearn_inverse(data,self.gamma)
        final_result = self.pca.standardization_inverse(inverse_kpca)
        return final_result

    def prepare_data(self,data):
        df = pd.DataFrame(data,columns=['component'])
        return df['component'].values

In [ ]:
def sample_first_prows(data, perc):
    return data.head(int(len(data)*(perc)))

In [ ]:
import numpy as np
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame)
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [ ]:
# convert series to supervised learning
def series_to_supervised_mimo(data, n_in, n_out, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [(df.columns[j]+'(t)') for j in range(n_vars)]
		else:
			names += [(df.columns[j]+'%d(t+%d)' % (j, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
df = pd.read_csv('E://Phd thesis//all experiments//MIMO//DRHFCM//IoT_datasets//energydata_complete.csv', sep=',')
df = df.drop(labels=['date','rv1', 'rv2'], axis=1)
df = clean_dataset(df)
df.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3
1,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2
2,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1
3,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0
4,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9


In [ ]:
reframed = series_to_supervised_mimo(df, 1, 1)
reframed.head()

,Appliances(t-1),lights(t-1),T1(t-1),RH_1(t-1),T2(t-1),RH_2(t-1),T3(t-1),RH_3(t-1),T4(t-1),RH_4(t-1),...,T8(t),RH_8(t),T9(t),RH_9(t),T_out(t),Press_mm_hg(t),RH_out(t),Windspeed(t),Visibility(t),Tdewpoint(t)
1,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2
2,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1
3,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0
4,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9
5,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.29,6.016667,734.0,92.0,5.333333,43.833333,4.8


In [ ]:
import math
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)
def sliding_window_pca(data,n_windows,train_size,transformation):

    result = {
         "window": [],
         "rmse": [],
         "nrmse": [],
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }

    final_result = {
         "window": [],
         "rmse": [],
         "nrmse": [],
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }

    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:

            print('-' * 20)
            print(f'training window {(ct)}')

            df_train = ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
            df_test = ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
            df_original = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)']

            models, data_train = Grouped_random_HFCM.run_train(df_train,transformation)
            forecast, data_test = Grouped_random_HFCM.run_test(models,df_test,transformation)


            df_forecats_columns = ['Appliances(t)', 'lights(t)', 'T1(t)', 'RH_1(t)', 'T2(t)', 'RH_2(t)',
                                   'T3(t)', 'RH_3(t)', 'T4(t)', 'RH_4(t)', 'T5(t)', 'RH_5(t)', 'T6(t)',
                                   'RH_6(t)', 'T7(t)', 'RH_7(t)', 'T8(t)', 'RH_8(t)', 'T9(t)', 'RH_9(t)',
                                   'T_out(t)', 'Press_mm_hg(t)', 'RH_out(t)', 'Windspeed(t)',
                                   'Visibility(t)', 'Tdewpoint(t)']
            columns = list(df_forecats_columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)

            for col in columns:
                m=Grouped_random_HFCM.create_GRHFCM_FTS(df_test)
                order=m.order
                ind=order-1
                print('order',order)
                if order==1:
                    original = df_original[col].values
                    forecast = df_forecast[col].values

                else:

                    original = df_original[col][:len(df_test)-ind]
#                     original = df_test[col][ind:]

                    original=original.values
                    forecast = df_forecast[col].values



#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label=col)
#                 ax.plot(forecast, label='Forecast')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()

                print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                rmse = round(mean_squared_error(original,forecast,squared=False),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                mape = round(mean_absolute_percentage_error(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["mape"].append(mape)
                result["window"].append(ct)
                result["variable"].append(col)

    measures = pd.DataFrame(result)
    return measures

In [ ]:
Grouped_random_HFCM=GRHFCM_PCA(num_components_pca =2 ,gamma = 0.9)

In [ ]:
data=reframed
kpca_result =  sliding_window_pca(data=data,n_windows=30,train_size=0.75,transformation='kPCA')

--------------------
training window 0
order 2
[ 12:37:58] getting statistics for variable: Appliances(t)
order 2
[ 12:37:58] getting statistics for variable: lights(t)
order 2
[ 12:37:58] getting statistics for variable: T1(t)
order 2
[ 12:37:58] getting statistics for variable: RH_1(t)
order 2
[ 12:37:58] getting statistics for variable: T2(t)
order 2
[ 12:37:58] getting statistics for variable: RH_2(t)
order 2
[ 12:37:58] getting statistics for variable: T3(t)
order 2
[ 12:37:58] getting statistics for variable: RH_3(t)
order 2
[ 12:37:58] getting statistics for variable: T4(t)
order 2
[ 12:37:58] getting statistics for variable: RH_4(t)
order 2
[ 12:37:58] getting statistics for variable: T5(t)
order 2
[ 12:37:58] getting statistics for variable: RH_5(t)
order 2
[ 12:37:58] getting statistics for variable: T6(t)
order 2
[ 12:37:58] getting statistics for variable: RH_6(t)
order 2
[ 12:37:58] getting statistics for variable: T7(t)
order 2
[ 12:37:58] getting statistics for variable:

<ipython-input-104-95f8cac1a97a>:8: RuntimeWarning: invalid value encountered in double_scalars
  return (rmse/x)


order 2
[ 12:38:10] getting statistics for variable: Appliances(t)
order 2
[ 12:38:10] getting statistics for variable: lights(t)
order 2
[ 12:38:10] getting statistics for variable: T1(t)
order 2
[ 12:38:10] getting statistics for variable: RH_1(t)
order 2
[ 12:38:10] getting statistics for variable: T2(t)
order 2
[ 12:38:10] getting statistics for variable: RH_2(t)
order 2
[ 12:38:10] getting statistics for variable: T3(t)
order 2
[ 12:38:10] getting statistics for variable: RH_3(t)
order 2
[ 12:38:10] getting statistics for variable: T4(t)
order 2
[ 12:38:10] getting statistics for variable: RH_4(t)
order 2
[ 12:38:10] getting statistics for variable: T5(t)
order 2
[ 12:38:10] getting statistics for variable: RH_5(t)
order 2
[ 12:38:10] getting statistics for variable: T6(t)
order 2
[ 12:38:10] getting statistics for variable: RH_6(t)
order 2
[ 12:38:10] getting statistics for variable: T7(t)
order 2
[ 12:38:10] getting statistics for variable: RH_7(t)
order 2
[ 12:38:10] getting st

<ipython-input-104-95f8cac1a97a>:8: RuntimeWarning: invalid value encountered in double_scalars
  return (rmse/x)


order 2
[ 12:39:09] getting statistics for variable: Appliances(t)
order 2
[ 12:39:09] getting statistics for variable: lights(t)
order 2
[ 12:39:09] getting statistics for variable: T1(t)
order 2
[ 12:39:09] getting statistics for variable: RH_1(t)
order 2
[ 12:39:09] getting statistics for variable: T2(t)
order 2
[ 12:39:09] getting statistics for variable: RH_2(t)
order 2
[ 12:39:09] getting statistics for variable: T3(t)
order 2
[ 12:39:09] getting statistics for variable: RH_3(t)
order 2
[ 12:39:09] getting statistics for variable: T4(t)
order 2
[ 12:39:09] getting statistics for variable: RH_4(t)
order 2
[ 12:39:09] getting statistics for variable: T5(t)
order 2
[ 12:39:09] getting statistics for variable: RH_5(t)
order 2
[ 12:39:09] getting statistics for variable: T6(t)
order 2
[ 12:39:09] getting statistics for variable: RH_6(t)
order 2
[ 12:39:09] getting statistics for variable: T7(t)
order 2
[ 12:39:09] getting statistics for variable: RH_7(t)
order 2
[ 12:39:09] getting st

In [ ]:
import statistics
#ttest = df.loc[:,:]

columns = list(reframed.loc[:,'Appliances(t)':'Tdewpoint(t)'].columns)

final_result = {
    "variable": [],
    "rmse": [],
    "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}

measures = kpca_result
var = measures.groupby("variable")

for col in columns:

    var_agr = var.get_group(col)

    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)

    print(f'Results: {(col,rmse,nrmse,mae,r2,mape)}')


final_measures_kpca = pd.DataFrame(final_result)

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('Appliances(t)', 65.178, 0.126, 31.106, 0.402, 0.289)
Results: ('lights(t)', 5.047, nan, 2.562, 0.184, 3912957698612111.0)
Results: ('T1(t)', 0.142, 0.072, 0.079, 0.922, 0.004)
Results: ('RH_1(t)', 0.5, 0.06, 0.251, 0.89, 0.006)
Results: ('T2(t)', 0.269, 0.071, 0.155, 0.929, 0.008)
Results: ('RH_2(t)', 0.514, 0.061, 0.304, 0.936, 0.008)
Results: ('T3(t)', 0.095, 0.054, 0.06, 0.946, 0.003)
Results: ('RH_3(t)', 0.299, 0.064, 0.173, 0.907, 0.004)
Results: ('T4(t)', 0.141, 0.07, 0.079, 0.916, 0.004)
Results: ('RH_4(t)', 0.264, 0.055, 0.161, 0.937, 0.004)
Results: ('T5(t)', 0.138, 0.067, 0.065, 0.906, 0.003)
Results: ('RH_5(t)', 1.998, 0.065, 0.747, 0.883, 0.013)
Results: ('T6(t)', 0.75, 0.076, 0.443, 0.922, 30285437644.034)
Results: ('RH_6(t)', 3.3, nan, 1.872, 0.905, 0.131)
Results: ('T7(t)', 0.109, 0.067, 0.058, 0.922, 0.003)
Results: ('RH_7(t)', 0.405, 0.059, 0.233, 0.946, 0.007)
Results: ('T8(t)', 0.124, 0.06, 0.07, 0.943, 0.003)
Results: ('RH_8(t)', 0.615, 0.068, 0.353, 0.93

,variable,rmse,nrmse,mae,r2,mape
0,Appliances(t),65.178,0.126,31.106,0.402,2.890000e-01
1,lights(t),5.047,NaN,2.562,0.184,3.912958e+15
2,T1(t),0.142,0.072,0.079,0.922,4.000000e-03
3,RH_1(t),0.500,0.060,0.251,0.890,6.000000e-03
4,T2(t),0.269,0.071,0.155,0.929,8.000000e-03
5,RH_2(t),0.514,0.061,0.304,0.936,8.000000e-03
6,T3(t),0.095,0.054,0.060,0.946,3.000000e-03
7,RH_3(t),0.299,0.064,0.173,0.907,4.000000e-03
8,T4(t),0.141,0.070,0.079,0.916,4.000000e-03
9,RH_4(t),0.264,0.055,0.161,0.937,4.000000e-03


In [ ]:

final_measures_kpca.to_csv("E:/Phd thesis/all experiments/MIMO/GRHFCM(must be chenged)/Apliances_lastversion/c=3/o=2/3concepts.csv",index = False, header=True)